<center><h2><b>CIC-Darknet2020</b></h2></center>
<center><h2><b>Feature selection</b></h2></center>

# Libraries importing and configuration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from darknet.feat_select import *

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.tree import plot_tree

In [2]:
random_state = 42
dataset_folder = "dataset/" 
img_folder = "imgs/"

np.random.seed(random_state)
plt.rcParams['figure.figsize'] = (18,6)
plt.rcParams["savefig.format"] = 'pdf'
plt.rcParams["savefig.dpi"] = 600
plt.rcParams.update({'font.size': 18})

In [3]:
samples_detec = pd.read_csv(dataset_folder + "darknet_dataset_processed_encoded.csv")
del samples_detec['Label.1']
samples_detec.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Benign
1,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Benign
2,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Benign
3,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Benign
4,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,1.437760e+15,3117718.131,1.437760e+15,1.437760e+15,Benign


In [4]:
samples_app = pd.read_csv(dataset_folder + "darknet_dataset_processed_encoded.csv")
samples_app = samples_app[samples_app['Label'] == 'Darknet']
del samples_app['Label']
samples_app.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label.1
117170,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,1.456330e+15,3.683488e+07,1.456330e+15,1.456330e+15,Audio-Streaming
117171,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,1.456330e+15,3.558961e+07,1.456330e+15,1.456330e+15,Audio-Streaming
117172,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,1.456330e+15,3.301057e+07,1.456330e+15,1.456330e+15,Audio-Streaming
117173,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,1.456330e+15,5.227810e+05,1.456330e+15,1.456330e+15,Audio-Streaming
117174,0,0,0,0,0,0,0,0,0,0,...,0.589647,0,0,0,0,1.456330e+15,1.052330e+05,1.456330e+15,1.456330e+15,Audio-Streaming


# Benign vs Darknet

In [5]:
X, y, labels = split_train_target(samples_detec, "Label")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
columns = samples_detec.columns.tolist()

## Decision tree

In [6]:
dt = DecisionTreeClassifier()
rfe_dt = select_features_rfecv(dt, X, y, 1, step=1)

In [ ]:
final_report_dt = final_evaluation_rfe(dt, X, y, labels, rfe_dt)
print()
print(f"Optimal number of features: {rfe_dt.n_features_}")
selected_columns_dt = summarize_feats(rfe_dt, dt, columns)

100%|██████████| 10/10 [00:46<00:00,  4.64s/it]


       precision       recall       f1-score       support       
Benign:   99.94		99.93		99.93		784910.00		
Darknet:  99.66		99.69		99.68		163010.00		


 10-fold Accuracy: 99.89%
Test accuracy: 99.91%


Confusion matrix:
[[38658    21]
 [   19  7991]]

Benign :    99.95%
Darknet :   99.74%

Optimal number of features: 86
Feature                   	Importance
------------------------------------------
col_91:                      	0.7628
col_49:                      	0.1205
Bwd Init Win Bytes:          	0.0420
col_24:                      	0.0410
hour:                        	0.0148
col_96:                      	0.0034
Idle Min:                    	0.0020
Idle Std:                    	0.0020
col_45:                      	0.0018
Idle Max:                    	0.0013
Flow IAT Min:                	0.0011
col_1:                       	0.0009
Src Port:                    	0.0007
FWD Init Win Bytes:          	0.0006
src_bogon:                   	0.0006
Packet Length Variance:      	0.0005
col

In [ ]:
dotfile = open("dtree3.dot", 'w')
export_graphviz(dt, out_file = dotfile, feature_names = selected_columns_dt)
dotfile.close()

## Random forest

In [ ]:
rf = RandomForestClassifier()
rfe_rf = select_features_rfecv(rf, X, y, 1, step=1)

Fitting estimator with 184 features.
Fitting estimator with 183 features.
Fitting estimator with 182 features.
Fitting estimator with 181 features.
Fitting estimator with 180 features.
Fitting estimator with 179 features.
Fitting estimator with 178 features.
Fitting estimator with 177 features.
Fitting estimator with 176 features.
Fitting estimator with 175 features.
Fitting estimator with 174 features.
Fitting estimator with 173 features.
Fitting estimator with 172 features.
Fitting estimator with 171 features.
Fitting estimator with 170 features.
Fitting estimator with 169 features.
Fitting estimator with 168 features.
Fitting estimator with 167 features.
Fitting estimator with 166 features.
Fitting estimator with 165 features.
Fitting estimator with 164 features.
Fitting estimator with 163 features.
Fitting estimator with 162 features.
Fitting estimator with 161 features.
Fitting estimator with 160 features.
Fitting estimator with 159 features.
Fitting estimator with 158 features.
F

In [ ]:
final_report_rf = final_evaluation_rfe(rf, X, y, labels, rfe_rf)
print()
print(f"Optimal number of features: {rfe_rf.n_features_}")
selected_columns_rf = summarize_feats(rfe_rf, rf, columns)

100%|██████████| 10/10 [01:22<00:00,  8.24s/it]


       precision       recall       f1-score       support       
Benign:   99.92		99.98		99.95		784910.00		
Darknet:  99.91		99.60		99.75		163010.00		


 10-fold Accuracy: 99.91%
Test accuracy: 99.95%


Confusion matrix:
[[38675     4]
 [   20  7990]]

Benign :    99.95%
Darknet :   99.95%

Optimal number of features: 28
Feature                   	Importance
------------------------------------------
col_91:                      	0.2297
col_96:                      	0.0976
col_76:                      	0.0821
col_49:                      	0.0809
col_58:                      	0.0730
col_24:                      	0.0614
src_ip_country:              	0.0446
Bwd Packet Length Min:       	0.0404
Fwd Header Length:           	0.0367
col_50:                      	0.0263
src_bogon:                   	0.0256
Flow Bytes/s:                	0.0242
col_9:                       	0.0183
Idle Max:                    	0.0160
Bwd Packet Length Mean:      	0.0155
col_21:                      	0.0150
hou

# Darknet traffic caracterization

In [ ]:
X1, y1, labels1 = split_train_target(samples_app, "Label.1")
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.33, random_state=random_state)
columns1 = samples_app.columns.tolist()

## Decision tree

In [ ]:
dt1 = DecisionTreeClassifier(random_state=random_state)
rfe_dt1 = select_features_rfecv(dt1, X1, y1, 1, step=1)

Fitting estimator with 184 features.
Fitting estimator with 183 features.
Fitting estimator with 182 features.
Fitting estimator with 181 features.
Fitting estimator with 180 features.
Fitting estimator with 179 features.
Fitting estimator with 178 features.
Fitting estimator with 177 features.
Fitting estimator with 176 features.
Fitting estimator with 175 features.
Fitting estimator with 174 features.
Fitting estimator with 173 features.
Fitting estimator with 172 features.
Fitting estimator with 171 features.
Fitting estimator with 170 features.
Fitting estimator with 169 features.
Fitting estimator with 168 features.
Fitting estimator with 167 features.
Fitting estimator with 166 features.
Fitting estimator with 165 features.
Fitting estimator with 164 features.
Fitting estimator with 163 features.
Fitting estimator with 162 features.
Fitting estimator with 161 features.
Fitting estimator with 160 features.
Fitting estimator with 159 features.
Fitting estimator with 158 features.
F

In [ ]:
final_evaluation_rfe(dt1, X1, y1, labels1, rfe_dt1)
print()
print(f"Optimal number of features: {rfe_dt1.n_features_}")
selected_columns_dt1 = summarize_feats(rfe_dt1, dt1, columns1)

100%|██████████| 10/10 [00:04<00:00,  2.49it/s]

               precision       recall       f1-score       support       
Audio-Streaming:  99.65		99.74		99.70		88830.00		
Browsing:         91.82		88.46		89.91		1750.00		
Chat:             99.25		99.15		99.20		30520.00		
Email:            95.16		95.19		95.15		3930.00		
File-Transfer:    98.22		98.21		98.21		17300.00		
P2P:              97.24		96.88		96.88		1570.00		
VOIP:             98.19		97.97		98.07		9860.00		
Video-Streaming:  96.65		96.93		96.77		9120.00		


 10-fold Accuracy: 98.94%
Test accuracy: 99.12%


Confusion matrix:
[[4388    2   11    0    0    0    0    0]
 [   5   80    0    0    1    1    1    0]
 [   4    4 1480    0    0    0    1    0]
 [   0    0    0  185    4    0    0    0]
 [   0    0    0   11  866    0    0    3]
 [   0    0    0    0    0   63    0    0]
 [   0    0    1    0    0    0  473    5]
 [   0    0    2    0    4    0   11  417]]

Audio-Streaming :   99.80%
Browsing :          93.02%
Chat :              99.06%
Email :             94.39%
File-Tr

In [ ]:
dotfile = open("dtree2.dot", 'w')
export_graphviz(dt1, out_file = dotfile, feature_names = selected_columns_dt1)
dotfile.close()

## Random forest

In [ ]:
rf1 = RandomForestClassifier(random_state=random_state)
rfe_rf1 = select_features_rfecv(rf1, X1, y1, 1, step=1)

Fitting estimator with 184 features.
Fitting estimator with 183 features.
Fitting estimator with 182 features.
Fitting estimator with 181 features.
Fitting estimator with 180 features.
Fitting estimator with 179 features.
Fitting estimator with 178 features.
Fitting estimator with 177 features.
Fitting estimator with 176 features.
Fitting estimator with 175 features.
Fitting estimator with 174 features.
Fitting estimator with 173 features.
Fitting estimator with 172 features.
Fitting estimator with 171 features.
Fitting estimator with 170 features.
Fitting estimator with 169 features.
Fitting estimator with 168 features.
Fitting estimator with 167 features.
Fitting estimator with 166 features.
Fitting estimator with 165 features.
Fitting estimator with 164 features.
Fitting estimator with 163 features.
Fitting estimator with 162 features.
Fitting estimator with 161 features.
Fitting estimator with 160 features.
Fitting estimator with 159 features.
Fitting estimator with 158 features.
F

In [ ]:
final_report_rf1 = final_evaluation_rfe(rf1, X1, y1, labels1, rfe_rf1)
print()
print(f"\nOptimal number of features: {rfe_rf1.n_features_}")
selected_columns_rf1 = summarize_feats(rfe_rf1, rf1, columns1)

100%|██████████| 10/10 [00:28<00:00,  2.87s/it]


               precision       recall       f1-score       support       
Audio-Streaming:  99.79		99.53		99.66		88830.00		
Browsing:         90.76		89.08		89.68		1750.00		
Chat:             99.25		99.54		99.39		30520.00		
Email:            96.79		96.46		96.56		3930.00		
File-Transfer:    98.17		98.38		98.27		17300.00		
P2P:              95.55		98.12		96.66		1570.00		
VOIP:             97.91		97.87		97.87		9860.00		
Video-Streaming:  96.13		97.04		96.57		9120.00		


 10-fold Accuracy: 98.97%
Test accuracy: 98.95%


Confusion matrix:
[[4377    3   13    0    3    1    3    1]
 [   3   79    3    0    1    0    1    1]
 [   7    3 1476    0    2    0    0    1]
 [   0    0    0  187    2    0    0    0]
 [   0    0    0    7  867    0    0    6]
 [   0    2    0    0    0   61    0    0]
 [   0    0    0    0    0    0  471    8]
 [   0    0    0    0    4    0    9  421]]

Audio-Streaming :   99.77%
Browsing :          90.80%
Chat :              98.93%
Email :             96.39%
File-Tr